                                            Holiday Planning Tool
This is a prototype of Hoiliday planning tool. The tool accepts the journey date and the prefered temperature of the customer as input and suggests two best travel packages for the customer based on the prefereced of the customer.

In [ ]:
#required libraries 
import http.client, urllib.request, urllib.parse, urllib.error, base64
import numpy
import matplotlib.pyplot as plt
import math
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import sklearn
import pandas as pd
import numpy as np
import glob
import warnings
from datetime import datetime
warnings.filterwarnings('ignore', '.*do not.*',)
from fbprophet import Prophet
import re

Function to search for the flights on the date of travel. Funtion accepts the travel destination airport code along with the date of travel as input and returns different possible flights to the destination aiport form New York airport.
The flight details are fetched using FlightLookup API.

In [215]:
def getFlightDetails(to,year,month,day):
    
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': '00c12c7f625f4eeaa65bac14b53489d7',
    }
    body=""

    params = urllib.parse.urlencode({
        # Request parameters
        'Connection': 'DIRECT'
    })

    try:
        conn = http.client.HTTPSConnection('flightlookup.azure-api.net')
        conn.request("GET", "/v1/xml/TimeTable/JFK/"+to+"/"+year+month+day+"/?%s" % params,body , headers)
        response = conn.getresponse()
        data = response.read()
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    #get flight timings
    dataString=str(data)
    flightDurations = re.findall('TotalFlightTime="PT(.+?)"', dataString)
    
    #get fliht names
    flights = re.findall('CompanyShortName="(.+?)" />', dataString)
    
    #store flight details into a pandas dataframe
    flightData = pd.DataFrame(
        {
         'Duration': flightDurations[0:2],
         'Flights': flights[0:2],
        })
    return(flightData)

Function implementing LSTM machine learning model for predicting weather data. Function accepts the path of the file containing the past weather data and writes the predicted weather data for the next 30 days into a csv file.

In [54]:
def forecaster_LSTM(filepath):
    print('...................LSTM forecasting......................')
    #get the dataset for prediction 
    dataset = pd.read_csv(filepath, usecols=[1], engine='python')
    df = dataset.values
    df = dataset.astype('float32')

    #scaling of input valued into the range of 0 to 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
 
    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    print(len(train), len(test))

    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return numpy.array(dataX), numpy.array(dataY)
    # reshape into X=t and Y=t+1
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # reshape input to be [samples, time steps, features]
    trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    

    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=10, batch_size=2, verbose=2)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    
    
    # calculate root mean squared error
    print("RMSE")
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
      
    #get future predictions
    futureX,futureY=[],[]
    for i in range(len(test)):
        a=test[i, 0]
        futureX.append([a])
    futureXP=numpy.array(futureX)
    futureXP = numpy.reshape(futureXP, (futureXP.shape[0], 1, futureXP.shape[1]))
    futurePredict=model.predict(futureXP)
    for i in range(30):
        futureX.append([futurePredict[-1]])
        futureXP=numpy.array(futureX)
        futureXP = numpy.reshape(futureXP, (futureXP.shape[0], 1, futureXP.shape[1]))
        futurePredict=model.predict(futureXP)
    futurePredict = scaler.inverse_transform(futurePredict)
    result=futurePredict[-30:]
    dates = pd.date_range(start='12/01/2017',end='12/30/2017',freq='D')
    predictedTemp=pd.DataFrame(dates,columns=['date'])
    predictedTemp['Temp']=result
    predictedTemp=predictedTemp.set_index(['date'])
    predictedTemp.to_csv('result/'+filepath, sep=',')
    

Function implementing Facebook Prophet for predicting weather data. Function accepts the path of the file containing the past weather data and writes the predicted weather data for the next 30 days into a csv file.

In [60]:
def forecast_Prophet(filepath):
    #read data
    df = pd.read_csv(filepath)
    df.columns=['ds', 'y']
    df['ds']=pd.DatetimeIndex(df['ds'])
    pd.to_numeric(df['y'], downcast='signed')
    
    #initialize and fit the model
    model=Prophet()
    model.fit(df)

    # make prediction for next 30 days
    future_dates = model.make_future_dataframe(periods=30)
    future_dates.tail()

    # Write the results into a file
    forecast = model.predict(future_dates)
    result=(forecast[['ds', 'yhat']].tail(30))
    result.columns=['date', 'Temp']
    result=result.set_index(['date'])
    result.to_csv('result/'+filepath, sep=',')

In [64]:
#predicts weather for the following 4 cities
#forecast_Prophet('Phoenix.csv')
#forecast_Prophet('Albuquerque.csv')
#forecast_Prophet('Portland.csv')
#forecast_Prophet('San Diego.csv')


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [55]:
#predict weather for Seattle
#forecaster_LSTM('Seattle.csv')

...................LSTM forecasting......................
1264 623
Epoch 1/10
 - 2s - loss: 0.0739
Epoch 2/10
 - 1s - loss: 0.0147
Epoch 3/10
 - 1s - loss: 0.0075
Epoch 4/10
 - 1s - loss: 0.0040
Epoch 5/10
 - 1s - loss: 0.0033
Epoch 6/10
 - 1s - loss: 0.0033
Epoch 7/10
 - 1s - loss: 0.0033
Epoch 8/10
 - 1s - loss: 0.0033
Epoch 9/10
 - 1s - loss: 0.0033
Epoch 10/10
 - 1s - loss: 0.0033
RMSE
Train Score: 1.87 RMSE
Test Score: 2.02 RMSE


Functoion which generates packages for the customers based on their preferences. Function accepts date of travel and prefered temperature of the customers as input and generated packages based on the input preferences

In [241]:
def package(date,temp):
    #read the files containing the predicted temperatures
    dfSeattle = pd.read_csv('result/Seattle.csv',index_col=0)
    dictSeattle=dfSeattle.to_dict()
    dfSan = pd.read_csv('result/San Diego.csv',index_col=0)
    dictSan=dfSan.to_dict()
    dfPortland = pd.read_csv('result/Portland.csv',index_col=0)
    dictPortland=dfPortland.to_dict()
    dfAlbuquerque = pd.read_csv('result/Albuquerque.csv',index_col=0)
    dictAlbuquerque=dfAlbuquerque.to_dict()
    dfPhoenix = pd.read_csv('result/Phoenix.csv',index_col=0)
    dictPhoenix=dfPhoenix.to_dict()
    
    #get temperatures of the five cities on the input date
    tempSeattle=dictSeattle["Temp"][date]
    tempSan=dictSan["Temp"][date]
    tempPortland=dictPortland["Temp"][date]
    tempAlbuquerque=dictAlbuquerque["Temp"][date]
    tempPhoenix=dictPhoenix["Temp"][date]
    
    #get differece of each temperature form the input temperature
    tempDiffSeattle=tempSeattle-temp
    tempDiffSan=tempSan-temp
    tempDiffPortland=tempPortland-temp
    tempDiffAlbuquerque=tempAlbuquerque-temp
    tempDiffPhoenix=tempPhoenix-temp
    
    #select the two cities with the least differeneces
    dictTempDiff={"Seattle":tempDiffSeattle, "San":tempDiffSan,"Portland":tempDiffPortland, "Albuquerque":tempDiffAlbuquerque,"Phoenix":tempDiffPhoenix}
    sortedDictTempDiff=sorted(dictTempDiff.values())
    for key,value in dictTempDiff.items():
        if(value==sortedDictTempDiff[0]):
            firstCity=key
        if(value==sortedDictTempDiff[1]):
            secondCity=key
    
    #airport codes of five cities
    dictAirportCodes={"Seattle":"SEA", "San":"SAN","Portland":"PDX", "Albuquerque":"ABQ","Phoenix":"PHX"}
    year=date[0:4]
    month=date[5:7]
    day=date[8:]
    
    #print the restults
    print("---------------------First Preferecne-----------------------------")
    print("City : ",firstCity)
    print("Temperature : ",(temp+dictTempDiff[firstCity]),"F")
    print("Flight Details :")
    print(getFlightDetails(dictAirportCodes[firstCity],'2019',month,day))
    
    print("\n\n---------------------Second Preferecne-----------------------------")
    print("City : ",secondCity)
    print("Temperature : ",(temp+dictTempDiff[secondCity]),"F")
    print("Flight Details :")
    print(getFlightDetails(dictAirportCodes[secondCity],'2019',month,day))
 

In [244]:
#call function to get the results
package("2017-12-25",250)

---------------------First Preferecne-----------------------------
City :  Albuquerque
Temperature :  275.6851411635925 F
Flight Details :
  Duration          Flights
0    5H12M  JetBlue Airways


---------------------Second Preferecne-----------------------------
City :  Seattle
Temperature :  276.52615 F
Flight Details :
  Duration            Flights
0    6H05M    Alaska Airlines
1    6H20M  American Airlines
